In [24]:
import os
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact

# Mahalanobis


        Mahalonobis distance is the distance between a point and a distribution. 
        And not between two distinct points. 
        It is effectively a multivariate equivalent of the Euclidean distance.

https://www.machinelearningplus.com/statistics/mahalanobis-distance/

The formula to compute Mahalanobis distance is as follows:

$D^2 = (x-m)^{T} \cdot C^{-1} \cdot (x-m)$

**Where,**


$D^2$ is the square of the Mahalanobis distance. 

$x$          is the vector of the observation (row in a dataset), 

$m$          is the vector of mean values of independent variables (mean of each column), 

$C^{-1}$     is the inverse covariance matrix of independent variables. 

$(x – m)$ is essentially the distance of the vector from the mean. We then divide this by the covariance matrix (or multiply by the inverse of the covariance matrix).

If you think about it, this is essentially a multivariate equivalent of the regular standardization (z = (x – mu)/sigma). That is, z = (x vector) – (mean vector) / (covariance matrix).

In [26]:
DATA_DIR = '/home/victor/Mestrado/Disciplinas/Visão Computacional/Projeto Final/bowfire/BowFire'
IMG_DIR = os.path.join(DATA_DIR, 'images')
MASK_DIR = os.path.join(DATA_DIR, 'masks')

In [46]:
list_imgs = [os.path.join(IMG_DIR, x) for x in os.listdir(IMG_DIR)]
list_masks = [os.path.join(MASK_DIR, x) for x in os.listdir(MASK_DIR)]

list_imgs = sorted(list_imgs)
list_masks = sorted(list_masks)

print('Total de imagens', len(list_imgs))
print('Total de máscaras', len(list_masks))

Total de imagens 119
Total de máscaras 119


In [50]:
@interact(count = (0, len(list_imgs)))
def view_images(count):
    global img
    global mask
    fig, ax = plt.subplots(1,2, figsize=(10,5))
    img = plt.imread(list_imgs[count])
    mask = plt.imread(list_masks[count])
    ax[0].imshow(img)
    ax[1].imshow(mask)
    
    for axs in ax:
        axs.axis('Off')

interactive(children=(IntSlider(value=59, description='count', max=119), Output()), _dom_classes=('widget-inte…

In [57]:
import numpy as np
import scipy.spatial.distance as SSD

covariance = np.cov([img.ravel()])
inv_cov = np.linalg.inv(covariance)  # inv. covariance matrix

def orig(A, B, C, M, inv_cov):
    h, w = A.shape
    result = np.zeros_like(A, dtype='float64')

    for i in range(h):
        for j in range(w):
            # array with particular pixels from each image
            v = np.array([A[i, j], B[i, j], C[i, j]])
            # calculate mahalanobis distance and insert value as a pixel
            result[i, j] = SSD.mahalanobis(v, M, inv_cov)
    return result

def using_cdist(A, B, C, M, inv_cov):
    D = np.dstack([A, B, C]).reshape(-1, 3)
    result = SSD.cdist(D, M[None, :], metric='mahalanobis', VI=inv_cov)
    result = result.reshape(A.shape)
    return result

expected = orig(A, B, C, M, inv_cov)
result = using_cdist(A, B, C, M, inv_cov)
assert np.allclose(result, expected)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional